In [7]:
import pyarrow as pa
import pyarrow.parquet as pq

dataset = pq.ParquetDataset('/Users/thomas/code/bigscience/pseudo_crawl/', filters=[('subset', '=', 'warc')])

dataset.schema

required group field_id=-1 hive_schema {
  optional int32 field_id=-1 id;
  optional binary field_id=-1 title (String);
  optional binary field_id=-1 link (String);
  optional binary field_id=-1 language (String);
  optional binary field_id=-1 url_surtkey_prefix (String);
  optional binary field_id=-1 url_surtkey (String);
  optional binary field_id=-1 url_host_tld (String);
  optional binary field_id=-1 url_host_registered_domain (String);
  optional binary field_id=-1 url_host_name (String);
  optional binary field_id=-1 url (String);
  optional int32 field_id=-1 fetch_status;
  optional int96 field_id=-1 fetch_time;
  optional binary field_id=-1 warc_filename (String);
  optional int32 field_id=-1 warc_record_offset;
  optional int32 field_id=-1 warc_record_length;
  optional binary field_id=-1 fetch_redirect (String);
  optional binary field_id=-1 content_mime_detected (String);
  optional binary field_id=-1 content_languages (String);
}

In [9]:
# (the two partitioning columns "crawl" and "subset" are included as well)
table = dataset.read(
    # First list of columns if from CC, Second from seed table
    columns=['id', 'url', 'warc_filename', 'content_mime_detected', 'content_languages'] + ["title", "link", "language"])


In [ ]:
table.shape


In [ ]:
# iterate over values of a single column
print(table.column_names)
print(len(table))
print(table[:5])

# convert to pandas DataFrame
df = table.to_pandas()

df.sample(5)

# DEBUG
df = df[:1000]

## Get PDF

We try to get all the pdfs

In [ ]:
# The url corresponds to the pdf url. Though we probably should be using the WARC record to select the pdf.
df["pdf_urls"] = [elt["url"] if elt['content_mime_detected'] == "application/pdf" else None for elt in df]

# Check results
pdf_df = df[df['content_mime_detected'] == "application/pdf"]
pdf_df

## Get HTML

## Get outgoing urls

## Aggregate everything into a single table

In [22]:
sub_df = df[:5]

# groupby = sub_df.groupby("link").agg(lambda grp: {"pdf_urls": [elt.url for elt in grp if grp["content_mime_detected"] == "application_pdf"]})

groupby = sub_df.groupby("link").agg({
    "title": lambda x: x[0],
    "link": lambda x: x[0],
    "pdf_urls": lambda x: x[0]
})

KeyError: "Column(s) ['pdf_urls'] do not exist"